In [3]:
import numpy as np
import pandas as pd

In [12]:
credits=pd.read_csv('tmdb_5000_credits.csv')
moves=pd.read_csv('tmdb_5000_movies.csv')

In [13]:
moves=moves.merge(credits,on='title')

In [14]:
moves.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [15]:
#genres
#id
#keywords
#title
#overview
#cast
#crew
moves=moves[['movie_id','title','overview','genres','keywords','cast','crew']]


In [16]:
moves.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [17]:
moves.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [18]:
moves.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [19]:
moves.dropna(inplace=True)
moves.duplicated().sum()

np.int64(0)

In [20]:
import ast
ast.literal_eval

<function ast.literal_eval(node_or_string)>

In [21]:
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [22]:
moves['genres']=moves['genres'].apply(convert)

In [23]:
moves['keywords']=moves['keywords'].apply(convert)

In [24]:
def convert2(obj):
  L=[]
  counter=0
  for i in ast.literal_eval(obj):
    if counter!=3:
      L.append(i['name'])
      counter+=1
    else:
      break
  return L


In [25]:
moves['cast']=moves['cast'].apply(convert2)

In [26]:
moves.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [27]:
def fetch_director(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
       L.append(i['name'])
       break
  return L

In [28]:
moves['crew']=moves['crew'].apply(fetch_director)

In [29]:
moves['overview']=moves['overview'].apply(lambda x:x.split())

In [30]:
moves['genres']=moves['genres'].apply(lambda x:[i.replace(" ","") for i in x])
moves['keywords']=moves['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
moves['cast']=moves['cast'].apply(lambda x:[i.replace(" ","") for i in x])
moves['crew']=moves['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [31]:
moves['tags']=moves['overview']+moves['genres']+moves['keywords']+moves['cast']+moves['crew']

In [32]:
new_data=moves[['movie_id','title','tags']]

In [33]:
new_data.head(2)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."


In [34]:
new_data['tags']=new_data['tags'].apply(lambda x:" ".join(x))

C:\Users\TYAGI\AppData\Local\Temp\ipykernel_4840\2257165486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags']=new_data['tags'].apply(lambda x:" ".join(x))


In [35]:
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [37]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)


In [38]:
new_data['tags']=new_data['tags'].apply(stem)

C:\Users\TYAGI\AppData\Local\Temp\ipykernel_4840\2662728569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags']=new_data['tags'].apply(stem)


In [39]:
new_data['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [40]:
new_data['tags']=new_data['tags'].apply(lambda x:x.lower())

C:\Users\TYAGI\AppData\Local\Temp\ipykernel_4840\3760292952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['tags']=new_data['tags'].apply(lambda x:x.lower())


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5600,stop_words='english')

In [46]:
vectors=cv.fit_transform(new_data['tags']).toarray()

In [47]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
similarity=cosine_similarity(vectors)

In [50]:
similarity[0]

array([1.        , 0.08134892, 0.08257228, ..., 0.0438529 , 0.        ,
       0.        ])

In [51]:
def recommend(move):
  move_index=new_data[new_data['title']==move].index[0]
  distances=similarity[move_index]
  moves_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in moves_list:
    print(new_data.iloc[i[0]].title)

In [52]:
recommend('Falcon Rising')

10th & Wolf
Street Kings
Sexy Beast
Compadres
Ladder 49


In [53]:
import pickle
pickle.dump(new_data,open('movies.pkl','wb'))

In [54]:
pickle.dump(new_data.to_dict(),open('movies_dict.pkl','wb'))

In [1]:
pickle.dump(similarity,open('similarity.pkl','wb'))

NameError: name 'pickle' is not defined

In [7]:
print("numpy",np.__version__)

numpy 2.0.1


In [8]:
print("pandas",pd.__version__)

pandas 2.2.2


In [13]:
print("sklearn",sklearn.__version__)

NameError: name 'sklearn' is not defined